# Prediction of Burn Area using Meteorological Data

### My goal with this kernel is to provide a reusable end-to-end Regression framework that is deployable in real world. It is imperative that the users adapt it to the dataset in hand and make small necessary changes, but the overall process and class-objects given can adapt to any regression system.

### Also, this kernel showcases how Scikit-Learn library can simplify and automate data preprocessing, model building and predicting for practical deployment.

### I chose this dataset because this problem showcases several fundamental challenges in data science, that are not always easy to overcome: - such as highly left-right skewed data, extreme outliers, underfitting, overfitting, lack of enough examples etc. Many of these problems can be dealt with by collecting more data, meaningful feature engineering, and may be removing rare cases or modelling with different techniques that can handle outliers much better, but that is not the motivation behind this kernel.

#### This dataset covers meteorological and spatiotemporal data for forest fires (between 2000 and 2003) in Portugal’s Montesinho Natural Park, with 13 attributes each for 517 such incidents. Our target attribute from these 13 is ‘area’ - total burned area in hectares (ha). The corresponding weather data is that which is registered by sensors when the fire was detected (or first broke out). We have: temp, RH (relative humidity), wind (speed), rain (accumulated precipitation over the last 30 minutes).

#### We also have DMC, DC, ISI and FFMC are components of the Canadian Forest Fire Weather Index (FWI) System, which measures the effects of fuel moisture and wind on fire behavior. These have been calculated using consecutive daily observations of temperature, relative humidity, wind speed, and 24-hour rainfall - i.e. these are time-lagged and not instantaneous values, unlike the four weather variables. Roughly, the higher these components, the more the expected severity of the fire.

#### Spatiotemporal data includes the month and day of the week that the incident occurred, and X and Y co-ordinates of the incident with respect to the park. Smaller fires are much more frequent. This is the case in this dataset, as well as with incidences of wildfires around the world, making this a difficult regression problem.



In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

#support python 2&3
from __future__ import division, print_function, unicode_literals

#common imports
import os
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

#make notebook's output stable across runs
np.random.seed(42)

#Pretty Figures
%matplotlib inline
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

#Ignore useless warnings
import warnings
warnings.filterwarnings(action='ignore',message='^internal gelsd')
pd.options.mode.chained_assignment = None
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
# function to load data
dataset_path = os.path.join("../input",'forestfires.csv')
def load_data(path):
        return pd.read_csv(dataset_path)
    
fire = load_data(dataset_path)
fire.head()

#### the dataset doesn't have any missing values, so I am randomly removing some data points from FFMC attribute to showcase how to handle missing data problems.


In [ ]:
fire.FFMC[np.random.choice(fire.index,15)] = np.nan
fire.info() #now we have 15 missing values for attribute FFMC

In [ ]:
fire.describe() #some attributes skewed to the right(mean>median) some to the left(mean<median)

In [ ]:
#let's name the categorical and numeical attributes 
categorical_attributes = list(fire.select_dtypes(include=['object']).columns)
numerical_attributes = list(fire.select_dtypes(include=['float64', 'int64']).columns)
print('categorical_attributes:', categorical_attributes)
print('numerical_attributes:', numerical_attributes)

In [ ]:
#Here we face an uniqe issue where a few months have only 1 or 2 data points. I chose to get rid of these so that the glm models
#can handle the tests properly. Let me know how you would have hanlded.
print('months', fire.month.value_counts(), sep='\n')
print('\n')
print('days', fire.day.value_counts(), sep='\n')

In [ ]:
months_to_remove = ['nov','jan','may']
forest_fire = fire.drop(fire[fire.month.isin(months_to_remove)].index ,axis=0)
forest_fire.month.value_counts()

In [ ]:
#visualizing distributions 
forest_fire.hist(bins=50, figsize=(15,10), ec='w')
plt.show()
#target-area-is heavily skewed, we have extreme outliers.

In [ ]:
plt.hist(forest_fire.area, ec='w', bins=100, color='red')
plt.text(800,100, 'max: '+str(forest_fire.area.max()), color='black', fontsize=14)
#Burnt area attribute ranges from 0 to 1091.

In [ ]:
#Grouping the the burnt area to get a better understanding
forest_fire['area_cat'] = pd.cut(forest_fire['area'], bins=[0,5, 10, 50, 100, 1100], include_lowest=True, 
                                 labels=['0-5', '5-10', '10-50', '50-100', '>100'])
forest_fire.area_cat.value_counts()

In [ ]:
#Interquartile range
Q1 = forest_fire.area.quantile(.25)
Q3 = forest_fire.area.quantile(.75)
IQR = 1.5*(Q3-Q1)
IQR

In [ ]:
#we are loosing quite a number of data points in already a small data set if we remove all outliers
forest_fire.query('(@Q1 - 1.5 * @IQR) <= area <= (@Q3 + 1.5 * @IQR)').area_cat.value_counts(sort=False)

#### I chose to remove any area greater than 100. You could do it differently, maybe with log transformations or simply removing every outlier example. It all depends how you state your problem. For me it is a matter of showcasing several tricks of Scikit-Learn, but I would have gone for a log-log model otherwise. Maybe in part2, where I will do a Regression framework using TensorFlow in eager.

In [ ]:
#remove outliers
forest_fire.drop(forest_fire[forest_fire.area>100].index,axis=0, inplace=True)
forest_fire.area_cat.value_counts()

#### Now the time for our assumptions. I remind you many assumptions we make during the iterative process might not make to the final model; As I already said, we make some assumptions, then during the model building process we might prove them right or wrong. But it always to helps to list the assumptions and take opinions of several domain experts but this is the luxury not every data scientist has.

#### Assumption1- Temperature might be a great indicator of wildfires and it might be the case that high temperature may lead to more area burnt.

In [ ]:
#Let's understand what temp ranges we have here.
plt.hist(forest_fire.temp, ec='w', bins=50, color='red')
plt.show()

In [ ]:
forest_fire['temp_bins'] = pd.cut(forest_fire.temp, bins=[0, 15, 20, 25, 40], include_lowest=True, 
                                 labels=['0-15', '15-20', '20-25', '>25'])
forest_fire.temp_bins.value_counts(sort=False)
#so we have from very cold 0-15 degrees to hot >25 degree.

#### As, I have assumed temp is a critical indicator, to have a golden test dataset I would prefer that both train and test has equally distributed temperature conditions. Stratified shuffle split will help us maintain the required strata compositions here. My strata set is temperature categories I have created above. Do note while creating groups make sure that all the groups are well represented.

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(forest_fire.values, forest_fire.temp_bins.values):
    st_train_set = forest_fire.iloc[train_index]
    st_test_set = forest_fire.iloc[test_index]

### Once you create the test data remember to not touch it until you finalize your model. Any exploration of test data would result in a biased perspective that might not generalize well to new data once the system is live.

In [ ]:
#this works like magic.
print(st_test_set.temp_bins.value_counts(sort=False)/len(st_test_set), sep='\n')
print(forest_fire.temp_bins.value_counts(sort=False)/len(forest_fire), sep='\n')

In [ ]:
#now lets drop the area and temp categories 
for _ in (st_train_set, st_test_set):
    _.drop(['area_cat','temp_bins'], axis=1, inplace=True)
    
forest_fire = st_train_set.copy()
forest_fire.head()

In [ ]:
#December had a few incidents but all on the higher burnt area side. Is it becuase dry weather or because more tourists? 
ax = plt.figure(figsize=(12,8))
ax = sns.boxplot(x='month', y='area', data=forest_fire, color='lightgrey', )
ax = sns.stripplot(x='month', y='area', data=forest_fire, color='red', jitter=0.4, size=4)

In [ ]:
#There are more incidents on weekends - Friday/Sat/Sun, it might mean that campers vactioning might have caused/spotted fires.
ax = plt.figure(figsize=(12,8))
ax = sns.boxplot(x='day', y='area', data=forest_fire, color='lightgrey', )
ax = sns.stripplot(x='day', y='area', data=forest_fire, color='red', jitter=0.4, size=4)

In [ ]:
#I am checking the temparature distribution as per the forest cordinates during the month of december
#it looks like the temps are low, so it could be a combinaton of dry weather and human made fire. But as data scientists,
#we must make important discoveries to arrive at any conclusive evidence. Else the above statments are just my mind made 
#fantsies
forest_fire[forest_fire.month=='dec'].plot(kind='scatter', x='X', y='Y', c='temp', cmap=plt.get_cmap('coolwarm'), colorbar=True)
plt.show()

#### Spearman rank correlation is more robust to the effect of outliers than Pearson’s correlations coefficient. If you need to know why I will load a new kernel sometime. But do remember we are just checking linear correlations here.

In [ ]:
corr_matrix = forest_fire.corr(method='spearman')
corr_matrix

In [ ]:
ax = plt.figure(figsize=(12,8))
ax = sns.heatmap(corr_matrix, cmap='PiYG')

In [ ]:
#corrleation with area
corr_matrix.area.sort_values(ascending=False)

In [ ]:
#visualizing relations of most related attributes
attributes = ['area', 'wind', 'temp', 'rain', 'RH']
sns.pairplot(forest_fire[attributes])
plt.show()

#### Not enough opportunity for feature engineering. we might take help of external data sources but need domain experts' input.

In [ ]:
#create a fresh copy of train to preprocess
forest_fire = st_train_set.drop('area', axis=1)
forest_fire_labels = st_train_set.area.copy()

#### I chose to remove coordinates X & Y before model building. The reason being here we are building a model that will use weather conditions to determine the severity of a wildfire based on area burnt, so keeping X&Y locations of one forest does ‘t generalize well to others.
#### Now in the process of data prep, you might need to create or drop several columns. So, it's a good idea to automate the process. Below an example of what an Attribute/Feature deleter will look like.

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

class AttributeDeleter(BaseEstimator, TransformerMixin):
    def __init__(self, delete=True):
        self.delete = delete
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        return np.delete(X,[fire.columns.get_loc(i) for i in['X','Y','area']],axis=1)
            

#### With same reasoning, you need to do a lot of transformations depending on the task, so having a custom pipeline that helps you automate these tasks will simplify your deployment process in the end.

#### To handle missing data, I chose to fill them with median values of the attribute you can choose to do differently, or once you finalize your model you can do a hyperparameter tuning to find out which imputing strategy works best using a grid search. Imputing simply means what value are you going to put in place of null values.

#### I chose to create of dummies out of my categorical attributes, but you can do a low-level vector representation of them using embeddings. OneHotEncoder does that for me.

#### And lastly, we have data attributes in many different scales. So, it's always a good idea to either standardize or normalize the scales. Reason being many instances based and model based learning have a distance based cost function to minimize to get the best possible similarity or model fit parameters. Having a standardized scale helps us find the best possible parameters in an efficient way.

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

categorical_attributes = list(forest_fire.select_dtypes(include=['object']).columns)
numerical_attributes = list(forest_fire.select_dtypes(include=['float64', 'int64']).columns)

num_pipeline = Pipeline([('imputer', SimpleImputer(strategy='median')),
                         ('drop_attributes', AttributeDeleter()),
                         ('std_scaler', StandardScaler()),
                        ])
full_pipeline = ColumnTransformer([('num', num_pipeline, numerical_attributes),
                                   ('cat', OneHotEncoder(), categorical_attributes),
                                  ])

train = full_pipeline.fit_transform(forest_fire)
train_labels = forest_fire_labels
train.shape ,forest_fire.shape

In [ ]:
#check the train data
train_df = pd.DataFrame(train, columns= numerical_attributes[2:] + list(full_pipeline.named_transformers_.cat.categories_[0]) +
             list(full_pipeline.named_transformers_.cat.categories_[1]))
train_df.head()

#### Now we can finally start the model selection process where we try out bunch of different models. Choose some and fine tune them before finalizing one.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

lin_reg = LinearRegression()
lin_reg.fit(train, train_labels)

area_predictions = lin_reg.predict(train)
lin_mse = mean_squared_error(train_labels, area_predictions)
lin_rmse = np.sqrt(lin_mse)
print('linear_train_rmse', lin_rmse) #model might be underfitting

from sklearn.model_selection import cross_val_score
scores = cross_val_score(lin_reg, train, train_labels, scoring='neg_mean_squared_error', cv=10)
lin_rmse_scores = np.sqrt(-scores)

def explain_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())
 
explain_scores(lin_rmse_scores)

In [ ]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor()
tree_reg.fit(train, train_labels)

area_predictions = tree_reg.predict(train)
tree_mse = mean_squared_error(train_labels, area_predictions)
tree_rmse = np.sqrt(tree_mse)
print('tree_train_rmse', tree_rmse) #model obviously overfitting

scores = cross_val_score(tree_reg, train, train_labels, scoring='neg_mean_squared_error', cv=10)
tree_rmse_scores = np.sqrt(-scores)
explain_scores(tree_rmse_scores)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf_reg = RandomForestRegressor()
rf_reg.fit(train, train_labels)

area_predictions = rf_reg.predict(train)
rf_mse = mean_squared_error(train_labels, area_predictions)
rf_rmse = np.sqrt(rf_mse)
print('rf_train_rmse', rf_rmse) #model is overfitting 

scores = cross_val_score(rf_reg, train, train_labels, scoring='neg_mean_squared_error', cv=10)
rf_rmse_scores = np.sqrt(-scores)
explain_scores(rf_rmse_scores)

In [ ]:
from sklearn.svm import SVR

svm_reg = SVR(kernel='linear')
svm_reg.fit(train, train_labels)

area_predictions = svm_reg.predict(train)
svm_mse = mean_squared_error(train_labels, area_predictions)
svm_rmse = np.sqrt(svm_mse)
print('svm_train_rmse', svm_rmse) #svm is generalizing well to crossvalidation set

scores = cross_val_score(svm_reg, train, train_labels, scoring='neg_mean_squared_error', cv=10)
svm_rmse_scores = np.sqrt(-scores)
explain_scores(svm_rmse_scores)

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

knn_reg = KNeighborsRegressor()
knn_reg.fit(train, train_labels)

area_predictions = knn_reg.predict(train)
knn_mse = mean_squared_error(train_labels, area_predictions)
knn_rmse = np.sqrt(knn_mse)
print('knn_train_rmse', knn_rmse) #overfiiting

scores = cross_val_score(knn_reg, train, train_labels, scoring='neg_mean_squared_error', cv=10)
knn_rmse_scores = np.sqrt(-scores)
explain_scores(knn_rmse_scores)

In [ ]:
#lets improve the models with hyperparameter tuning

from sklearn.model_selection import GridSearchCV

param_grid = [{'bootstrap':[False,True],'n_estimators':[75,100,125,150,200], 'max_features':[1,2,4,6]}]

forest_reg = RandomForestRegressor()

grid_search = GridSearchCV(forest_reg, param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True)

grid_search.fit(train, train_labels)

In [ ]:
grid_search.best_params_ 

In [ ]:
grid_search.best_estimator_

In [ ]:
feature_importances = grid_search.best_estimator_.feature_importances_
attributes = numerical_attributes + list(full_pipeline.named_transformers_.cat.categories_[0]) +\
             list(full_pipeline.named_transformers_.cat.categories_[1])
    
sorted(zip(feature_importances, attributes), reverse=True)

In [ ]:
indices = np.argsort(feature_importances)
plt.figure(figsize=(12,8))
plt.title('Feature Importances')
plt.barh(range(len(indices)), feature_importances[indices], color='lightblue', align='center')
plt.yticks(range(len(indices)), [attributes[i] for i in indices])
plt.xlabel('Relative Importance')
plt.show()

In [ ]:
#lets try with RandomizedSearchCV

from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_dist = {'n_estimators': randint(low=10, high=250),
              'max_features': randint(low=1, high=24),
             }

forest_reg = RandomForestRegressor(random_state=42)
rnd_search = RandomizedSearchCV(forest_reg, param_dist, n_iter=40, cv=5, scoring='neg_mean_squared_error', random_state=42)

rnd_search.fit(train, train_labels)


In [ ]:
rnd_search.best_params_

In [ ]:
np.sqrt(-rnd_search.best_score_), np.sqrt(-grid_search.best_score_) #both have very similar scores even though they came up with 
#different best parameters. which one to use, I leave to you explain

In [ ]:
#tuning svr

param_grid = [{'kernel': ['linear'], 'C':[0.5,1,5,10,30]},
              {'kernel':['rbf'], 'C':[5,10,15,20], 'gamma':[0.5,1.0,1.5,2.0]},
             ]

svm_reg = SVR()
grid_search = GridSearchCV(svm_reg, param_grid, cv=5, scoring='neg_mean_squared_error', verbose=1, n_jobs=4)
grid_search.fit(train,train_labels)

negative_mse = grid_search.best_score_
rmse = np.sqrt(-negative_mse)
rmse

In [ ]:
grid_search.best_params_

In [ ]:
from scipy.stats import expon, reciprocal

param_dist = {'kernel':['linear','rbf'],
                  'C':reciprocal(1,100),
                  'gamma':expon(scale=1.0)}
svm_reg = SVR()
rnd_search = RandomizedSearchCV(svm_reg, param_distributions=param_dist, n_iter=100, cv=5,
                               scoring='neg_mean_squared_error', verbose=2, n_jobs=4, random_state=42)
rnd_search.fit(train, train_labels)

negative_mse = rnd_search.best_score_
rmse = np.sqrt(-negative_mse)
rmse

In [ ]:
rnd_search.best_params_ #randomized search is able to find better parameters for rbf kernel in same number of iterations

#### At times it is a good idea to use only your best features in the model, but it is context dependent. Below sample code helps you add another transformer to your pipeline that chooses the best feature based on an estimated feature importance

In [ ]:
def indices_top_features(impotance, top):
    return np.sort(np.argpartition(np.array(impotance), -top)[-top:])

class TopFeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, feature_importnaces, top):
        self.feature_importnaces = feature_importances
        self.top = top
    def fit(self, X, y=None):
        self.feature_indcies_ = indices_top_features(self.feature_importnaces, self.top)
        return self
    def transform(self,X):
        return X[:, self.feature_indcies_]

In [ ]:
data_prep_feature_seletion_pipe = Pipeline([('prep', full_pipeline),
                                            ('fe_select', TopFeatureSelector(feature_importances,5)) #here am choosing top 5 features you can choose others depending on 
                                           ])                                                        #on what you want to keep  

In [ ]:
train_fe_selected = data_prep_feature_seletion_pipe.fit_transform(forest_fire)
train_fe_selected.shape

In [ ]:
#now let's try knn with these reduced dimensions 

knn_reg = KNeighborsRegressor()
knn_reg.fit(train_fe_selected, train_labels)

area_predictions = knn_reg.predict(train_fe_selected)
knn_mse = mean_squared_error(train_labels, area_predictions)
knn_rmse = np.sqrt(knn_mse)
print('knn_train_rmse', knn_rmse) #knn is generalizing well to crossvalidation set

scores = cross_val_score(knn_reg, train_fe_selected, train_labels, scoring='neg_mean_squared_error', cv=10)
knn_rmse_scores = np.sqrt(-scores)
explain_scores(knn_rmse_scores)

In [ ]:
#lets tune KNN 
param_grid = {'weights': ['uniform', 'distance'], 'n_neighbors': list(range(1,36,5))}

knn_reg= KNeighborsRegressor()
knn_grid_search = GridSearchCV(knn_reg, param_grid, cv=5, scoring='neg_mean_squared_error', verbose=1, n_jobs=4)
knn_grid_search.fit(train_fe_selected,train_labels)

knn_negative_mse = knn_grid_search.best_score_
knn_rmse = np.sqrt(-knn_negative_mse)
knn_rmse

In [ ]:
#and for final model I want to try GBM
from xgboost import XGBRegressor

xgb_reg = XGBRegressor()
xgb_reg.fit(train_fe_selected, train_labels)

area_predictions = xgb_reg.predict(train_fe_selected)
xgb_mse = mean_squared_error(train_labels, area_predictions)
xgb_rmse = np.sqrt(xgb_mse)
print('xgb_train_rmse', xgb_rmse) #overfitting

scores = cross_val_score(xgb_reg, train_fe_selected, train_labels, scoring='neg_mean_squared_error', cv=10)
xgb_rmse_scores = np.sqrt(-scores)
explain_scores(xgb_rmse_scores)

In [ ]:
param_grid = {'objective':['reg:linear'],
              'learning_rate': [0.02,0.03,0.04], 
              'max_depth': [1,2],
              'min_child_weight': [2,3,4],
              'silent': [1],
              'subsample': [0.8],
              'colsample_bytree': [0.2,0.3,0.4],
              'n_estimators': [50,60,70,100]}

xgb_reg = XGBRegressor()

xgb_grid_search = GridSearchCV(xgb_reg, param_grid, cv=5, scoring='neg_mean_squared_error', verbose=1, n_jobs=4)
xgb_grid_search.fit(train_fe_selected,train_labels)

xgb_negative_mse = grid_search.best_score_
xgb_rmse = np.sqrt(-xgb_negative_mse)
xgb_rmse

In [ ]:
xgb_grid_search.best_params_

In [ ]:
final_model = knn_grid_search.best_estimator_

X_test = st_test_set.drop(['area'], axis=1)
y_test = st_test_set['area'].copy()

X_test_prepared = data_prep_feature_seletion_pipe.transform(X_test)

final_predictions = final_model.predict(X_test_prepared)

final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse)
final_rmse

### Let's say, I determined to deploy XGB model. I will add a predictor to my previously defined Pipeline as the last element., which will take care of the entire process from preparing the data to predicting area.

In [ ]:
prepare_select_and_predict_pipeline = Pipeline([
    ('preparation', full_pipeline),
    ('feature_selection', TopFeatureSelector(feature_importances, 5)),
    ('final_model', XGBRegressor(**xgb_grid_search.best_params_))
])

In [ ]:
prepare_select_and_predict_pipeline.fit(forest_fire,forest_fire_labels)

In [ ]:
final_predictions = prepare_select_and_predict_pipeline.predict(X_test)

### An example showing a small use case. Let's assume some_data here are the new data that our system will predict for.

In [ ]:
some_data = forest_fire[:4]
some_labels = forest_fire_labels[:4]

print("Predictions:\t", prepare_select_and_predict_pipeline.predict(some_data))
print("Labels:\t\t", list(some_labels))

In [ ]:
#Confidence Interval of our Predictions will help us better understand the ouput of our model
from scipy import stats

confidence = 0.95
squared_errors = (final_predictions-y_test)**2
np.sqrt(stats.t.interval(confidence, len(squared_errors)-1, loc=squared_errors.mean(), scale=stats.sem(squared_errors)))